# Network, Graph and Graph Database

In [ ]:
COLAB = False

## Prepare the dataset

In [3]:
# https://gist.github.com/korakot/328aaac51d78e589b4a176228e4bb06f
    
if COLAB:
    # Download from Kaggle - https://www.kaggle.com/general/74235
    
    ! pip install -q kaggle
    
    from google.colab import files
    files.upload()    # Upload kaggle.json
    
    !mkdir ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    
    # ! kaggle datasets list
    # ! kaggle competitions download -c 'name-of-competition'
else:
    # Use local copy
    pass